In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
pip install openpyxl

In [1]:
pip install xlrd

![](http://sahinuzunsimsek.com/wp-content/uploads/2020/11/AB-Testing.png)

**AB Testi ile Bidding Yöntemlerinin Dönüşümünün Karşılaştırılması**

Facebook kısa süre önce mevcut "maximum bidding (maksimum teklif verme)" adı verilen teklif verme türüne alternatif olarak yeni bir teklif türü olan "average bidding (ortalama teklif verme)" i tanıttı.
Müşterilerimizden biri olan bombabomba.com, bu yeni özelliği test etmeye karar verdi ve average bidding'in
maximum bidding'den daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B testi yapmak istiyor.

* Bu A/B testinde, melihkomutgan.com hedef kitlesini rastgele eşit büyüklükte iki gruba ayırır, kontrol ve test grubu.
"maximum bidding" olan bir Facebook reklam kampanyası "Control Group" a sunulur ve "average bidding" olan başka bir
kampanya "Test Group" a sunulur.

* A/B testi 1 aydır devam ediyor ve melihkomutgan.com şimdi sizden bu A/B testinin sonuçlarını analiz etmenizi bekliyor.

* melihkomutgan.com için nihai başarı ölçütü Purchase 'dır. Bu nedenle, istatistiksel testler için purchase metriğine odaklanmalıyız.

**Veri Seti:**
Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin
yanı sıra buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır.

* impression : Reklam görüntüleme sayısı
* Click : Görüntülenen reklama tıklama sayısı
* Purchase : Tıklanan reklamlar sonrası satın alınan ürün sayısı
* Earning : Satın alınan ürünler sonrası elde edilen kazanç

In [1]:
# Gerekli Kütüphanelerin Yüklenmesi
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import shapiro, ttest_ind
import scipy.stats as stats
pd.set_option('display.max_columns', None)

In [1]:
Control_Group = pd.read_excel("../input/ab-testing-dataset/ab_testing.xlsx", sheet_name='Control Group')  # maximum bidding


In [1]:
Control_Group.head()

In [1]:
Control_Group.shape

In [1]:
Control_Group.isnull().sum()

In [1]:
Control_Group.describe([0.90,0.95,0.99]).T

In [1]:
Test_Group = pd.read_excel("../input/ab-testing-dataset/ab_testing.xlsx", sheet_name='Test Group') # average bidding


In [1]:
Test_Group.head()

In [1]:
Test_Group.shape

In [1]:
Test_Group.isnull().sum()

In [1]:
Test_Group.describe([0.90,0.95,0.99]).T

**AB Testing (Bağımsız İki Örneklem T Testi)**

1. Hipotezleri Kur
2. Varsayım Kontrolü
- 1. Normallik Varsayımı (shapiro)
2. Varyans Homojenliği (levene)
3. Hipotezin Uygulanması
   - 1. Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi (parametrik test)
   - 2. Varsayımlar sağlanmıyorsa mannwhitneyu testi (non-parametrik test)
4. p-value değerine göre sonuçları yorumla

 Not:
 - Normallik sağlanmıyorsa direk 2 numara. Varyans homojenliği sağlanmıyorsa 1 numaraya arguman girilir.
 - Normallik incelemesi öncesi aykırı değer incelemesi ve düzeltmesi yapmak faydalı olabilir.

In [1]:
# H0: "Maximum Bidding" kampanyası sunulan Kontrol grubu ile "Average Bidding" kampanyası sunulan
#      Test grubunun satın alma sayılarının ortalaması arasında istatistiksel anlamlı bir fark yoktur.

# H1: "Maximum Bidding" kampanyası sunulan Kontrol grubu ile "Average Bidding" kampanyası sunulan
#      Test grubunun satın alma sayılarının ortalaması arasında istatistiksel anlamlı bir fark vardır.

print(" Mean of purchase of control group: %.3f"%Control_Group["Purchase"].mean(),"\n",
      "Mean of purchase of test group: %.3f"%Test_Group["Purchase"].mean())

groupA = Control_Group["Purchase"]
groupB = Test_Group["Purchase"]

In [1]:
# 1. Normallik Varsayımı
# veri setinde medyan değerin ve ortalamanın birbirine çok yakın olması durumudur.
# Yani verilerin çoğunluğu ortalama değer/medyan etrafında kümelenir ve bir tepecik oluşturur.
# Grafiğin simetrik olması en önemli özelliğidir.


# H0: Normal dağılım varsayımı sağlanmaktadır.
# H1:..sağlanmamaktadır.

test_istatistigi, pvalue = shapiro(groupA)
print('Test İstatistiği = %.4f, p-değeri = %.4f' % (test_istatistigi, pvalue))
# p-value < ise 0.05 'ten HO RED.
# p-value < değilse 0.05 H0 REDDEDILEMEZ.
# Normal dağılım sağlanmakta

test_istatistigi, pvalue = shapiro(groupB)
print('Test İstatistiği = %.4f, p-değeri = %.4f' % (test_istatistigi, pvalue))
# p-value < ise 0.05 'ten HO RED.
# p-value < değilse 0.05 H0 REDDEDILEMEZ.
# Normal dağılım sağlanmakta

In [1]:
# İki grubun dağılımlarının birbirine benzer olup olmaması

# H0: Varyanslar Homojendir
# H1: Varyanslar Homojen Değildir

test_istatistigi, pvalue = stats.levene(groupA,groupB)
print('Test İstatistiği = %.4f, p-değeri = %.4f' % (test_istatistigi, pvalue))
# p-value < ise 0.05 'ten HO RED.
# p-value < değilse 0.05 H0 REDDEDILEMEZ.
# Varyanslar homejendir.

* Hipotezin Uygulanması

In [1]:
# H0: "Maximum Bidding" kampanyası sunulan Kontrol grubu ile "Average Bidding" kampanyası sunulan
#      Test grubunun satın alma sayılarının ortalaması arasında istatistiksel anlamlı bir fark yoktur.

# H1: "Maximum Bidding" kampanyası sunulan Kontrol grubu ile "Average Bidding" kampanyası sunulan
#      Test grubunun satın alma sayılarının ortalaması arasında istatistiksel anlamlı bir fark vardır.

# Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi (parametrik test)

test_stat, pvalue = ttest_ind(groupA,groupB,equal_var=True)
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# P değeri 0,05'ten büyük olduğundan H0 reddedilmez. Dolayısıyla, “maksimum teklif” kampanyası sunulan Kontrol grubu ile
# “ortalama teklif” kampanyası sunulan Test grubu arasında istatistiksel olarak anlamlı bir fark yoktur.

* Bağımsız t-testi kullandık çünkü birbirinden bağımsız  iki grubun ortalamaları arasında belirli özelliklerde ilişkili olabilecek anlamlı bir fark olup olmadığını belirlemek istiyoruz ve varsayımlara sonucunda iki grubunda normal dağılımave varyans homojenliğine sahip olduğunu gördük.

* Satın alma anlamında anlamlı bir fark olmadığından müşteri iki yöntemden birini seçebilir fakat burada diğer istatistiklerdeki farklılıklar da önem arz edecektir. Tıklanma, Etkileşim, Kazanç ve Dönüşüm Oranlarındaki farklılıklar değerlendirilip hangi yöntemin daha kazançlı olduğu tespit edilebilir. Test'in süresini uzatabiliriz.